In [20]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score
np.random.seed(42)

from matplotlib import style
style.use('fivethirtyeight')

In [21]:
data_dir = "C:\\Users\\Team Knowhow\\Documents\\YEAR 4\\Data Science Toolbox\\DST-NN-Project\\Erin Pollard"
train_path = "C:\\Users\\Team Knowhow\\Documents\\YEAR 4\\Data Science Toolbox\\DST-NN-Project\\Erin Pollard\\Train"
test_path = "C:\\Users\\Team Knowhow\\Documents\\YEAR 4\\Data Science Toolbox\\DST-NN-Project\\Erin Pollard\\Test"

# Resizing the images to 30x30x3
IMG_HEIGHT = 30
IMG_WIDTH = 30
channels = 3

In [22]:
NUM_CATEGORIES = len(os.listdir(train_path))
NUM_CATEGORIES = NUM_CATEGORIES 
NUM_CATEGORIES

43

In [23]:
# Label Overview
classes = { 0:'Speed limit (20km/h)',
            1:'Speed limit (30km/h)', 
            2:'Speed limit (50km/h)', 
            3:'Speed limit (60km/h)', 
            4:'Speed limit (70km/h)', 
            5:'Speed limit (80km/h)', 
            6:'End of speed limit (80km/h)', 
            7:'Speed limit (100km/h)', 
            8:'Speed limit (120km/h)', 
            9:'No passing', 
            10:'No passing veh over 3.5 tons', 
            11:'Right-of-way at intersection', 
            12:'Priority road', 
            13:'Yield', 
            14:'Stop', 
            15:'No vehicles', 
            16:'Veh > 3.5 tons prohibited', 
            17:'No entry', 
            18:'General caution', 
            19:'Dangerous curve left', 
            20:'Dangerous curve right', 
            21:'Double curve', 
            22:'Bumpy road', 
            23:'Slippery road', 
            24:'Road narrows on the right', 
            25:'Road work', 
            26:'Traffic signals', 
            27:'Pedestrians', 
            28:'Children crossing', 
            29:'Bicycles crossing', 
            30:'Beware of ice/snow',
            31:'Wild animals crossing', 
            32:'End speed + passing limits', 
            33:'Turn right ahead', 
            34:'Turn left ahead', 
            35:'Ahead only', 
            36:'Go straight or right', 
            37:'Go straight or left', 
            38:'Keep right', 
            39:'Keep left', 
            40:'Roundabout mandatory', 
            41:'End of no passing', 
            42:'End no passing veh > 3.5 tons' }

In [24]:
image_data = []
image_labels = []

for i in range(NUM_CATEGORIES):
    path = data_dir + '/Train/' + str(i)
    images = os.listdir(path)

    for img in images:
        try:
            image = cv2.imread(path + '/' + img)
            image_fromarray = Image.fromarray(image, 'RGB')
            resize_image = image_fromarray.resize((IMG_HEIGHT, IMG_WIDTH))
            image_data.append(np.array(resize_image))
            image_labels.append(i)
        except:
            print("Error in " + img)

# Changing the list to numpy array
image_data = np.array(image_data)
image_labels = np.array(image_labels)

print(image_data.shape, image_labels.shape)

(39209, 30, 30, 3) (39209,)


In [35]:
image_data.shape

(39209, 30, 30, 3)

In [25]:
shuffle_indexes = np.arange(image_data.shape[0])
np.random.shuffle(shuffle_indexes)
image_data = image_data[shuffle_indexes]
image_labels = image_labels[shuffle_indexes]

In [34]:
image_data

array([[[[ 42,  59, 106],
         [ 39,  54,  95],
         [ 33,  50,  74],
         ...,
         [ 41,  82, 139],
         [ 47,  82, 131],
         [ 65, 109, 170]],

        [[ 47,  66, 105],
         [ 39,  58,  91],
         [ 29,  50,  88],
         ...,
         [ 25,  59,  95],
         [ 18,  48,  61],
         [ 27,  65,  79]],

        [[ 49,  72, 116],
         [ 37,  65,  98],
         [ 28,  58, 101],
         ...,
         [ 31,  62,  93],
         [ 24,  54,  74],
         [ 21,  55,  74]],

        ...,

        [[ 30,  51,  94],
         [ 36,  55,  90],
         [ 30,  48,  82],
         ...,
         [ 33,  68,  94],
         [ 38,  66,  99],
         [ 39,  70,  99]],

        [[ 42,  60,  97],
         [ 43,  58, 100],
         [ 50,  63, 105],
         ...,
         [ 41,  74, 104],
         [ 55,  80, 116],
         [ 44,  71, 104]],

        [[ 52,  77, 115],
         [ 49,  73, 122],
         [ 54,  78, 123],
         ...,
         [ 46,  83, 113],
        

In [26]:
X_train, X_val, y_train, y_val = train_test_split(image_data, image_labels, test_size=0.3, random_state=42, shuffle=True)

X_train = X_train/255 
X_val = X_val/255

print("X_train.shape", X_train.shape)
print("X_valid.shape", X_val.shape)
print("y_train.shape", y_train.shape)
print("y_valid.shape", y_val.shape)

X_train.shape (27446, 30, 30, 3)
X_valid.shape (11763, 30, 30, 3)
y_train.shape (27446,)
y_valid.shape (11763,)


In [27]:
y_train = keras.utils.to_categorical(y_train, NUM_CATEGORIES)
y_val = keras.utils.to_categorical(y_val, NUM_CATEGORIES)

print(y_train.shape)
print(y_val.shape)

(27446, 43)
(11763, 43)


In [28]:
model = keras.models.Sequential([    
    keras.layers.Conv2D(filters=16, kernel_size=(3,3), activation='relu', input_shape=(IMG_HEIGHT,IMG_WIDTH,channels)),
    keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu'),
    keras.layers.MaxPool2D(pool_size=(2, 2)),
    keras.layers.BatchNormalization(axis=-1),
    
    keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    keras.layers.Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
    keras.layers.MaxPool2D(pool_size=(2, 2)),
    keras.layers.BatchNormalization(axis=-1),
    
    keras.layers.Flatten(),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(rate=0.5),
    
    keras.layers.Dense(43, activation='softmax')
])

In [29]:
lr = 0.001
epochs = 1

opt = tf.keras.optimizers.legacy.Adam(lr=lr, decay=lr / (epochs * 0.5))
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

C:\Users\Team Knowhow\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [30]:
aug = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.15,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode="nearest")

history = model.fit(aug.flow(X_train, y_train, batch_size=32), epochs=epochs, validation_data=(X_val, y_val))

858/858 [==============================] - 73s 82ms/step - loss: 1.1147 - accuracy: 0.7030 - val_loss: 0.0640 - val_accuracy: 0.9832


In [31]:
test = pd.read_csv(data_dir + '/Test.csv')

labels = test["ClassId"].values
imgs = test["Path"].values

data =[]

for img in imgs:
    try:
        image = cv2.imread(data_dir + '/' +img)
        image_fromarray = Image.fromarray(image, 'RGB')
        resize_image = image_fromarray.resize((IMG_HEIGHT, IMG_WIDTH))
        data.append(np.array(resize_image))
    except:
        print("Error in " + img)
X_test = np.array(data)
X_test = X_test/255

pred = np.argmax(model.predict(X_test), axis=-1)

#Accuracy with the test data
print('Test Data accuracy: ',accuracy_score(labels, pred)*100)

395/395 [==============================] - 7s 17ms/step
Test Data accuracy:  94.43388756927949


In [36]:
labels

array([16,  1, 38, ...,  6,  7, 10], dtype=int64)

In [32]:
from sklearn.metrics import confusion_matrix
cf = confusion_matrix(labels, pred)

In [33]:
from sklearn.metrics import classification_report

print(classification_report(labels, pred))

              precision    recall  f1-score   support

           0       0.93      0.70      0.80        60
           1       0.95      0.98      0.97       720
           2       0.97      0.97      0.97       750
           3       0.95      0.90      0.92       450
           4       0.96      0.97      0.96       660
           5       0.80      0.96      0.87       630
           6       0.97      0.89      0.93       150
           7       0.98      0.80      0.88       450
           8       0.94      0.98      0.96       450
           9       0.97      0.99      0.98       480
          10       0.99      0.97      0.98       660
          11       0.99      0.81      0.89       420
          12       0.99      1.00      0.99       690
          13       0.99      1.00      0.99       720
          14       1.00      1.00      1.00       270
          15       1.00      1.00      1.00       210
          16       0.97      1.00      0.98       150
          17       1.00    